In [1]:
#As always, lets start by importing the libraries we need
import json    
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import helper

#And the tools from Sklearn to do our clustering
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import mean_squared_error

from scipy.sparse import csr_matrix

from datetime import datetime


#To make things cleaner, lets also not display all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
businesses = []
with open('business.json') as f:
    for line in f:
        businesses.append(json.loads(line))
        
reviews = []
with open('review.json') as f:
    for line in f:
        reviews.append(json.loads(line))
        
testBusiness = businesses[1]
testReview = reviews[1]

print(testBusiness)
print('')
print(testReview)


{'business_id': 'QXAEGFB4oINsVuTFxEYKFQ', 'name': 'Emerald Chinese Restaurant', 'address': '30 Eglinton Avenue W', 'city': 'Mississauga', 'state': 'ON', 'postal_code': 'L5R 3E7', 'latitude': 43.6054989743, 'longitude': -79.652288909, 'stars': 2.5, 'review_count': 128, 'is_open': 1, 'attributes': {'RestaurantsReservations': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'Caters': 'True', 'NoiseLevel': "u'loud'", 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'False', 'BikeParking': 'False', 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'HasTV': 'False', 'WiFi': "u'no'", 'GoodForKids': 'True', 'Alcohol': "u

In [3]:
#Put all data into dataframes 

review_id = []
user_id = []
business_id = []
stars = []
date = []

for review in reviews:
    review_id.append(review['review_id'])
    user_id.append(review['user_id'])
    business_id.append(review['business_id'])
    stars.append(review['stars'])
    date.append(review['date'])

ratingsDF = pd.DataFrame({'review_id': review_id,
                             'user_id': user_id,
                             'business_id': business_id,
                         'stars':stars,
                         'date':date})
ratingsDF

,review_id,user_id,business_id,stars,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38
...,...,...,...,...,...
6685895,0pCaq1hqAViS2N6ldlp_sQ,Xu_YazIajfsc7SRj1wJ6Mg,RXBFk3tVBxiTf3uOt9KExQ,5.0,2018-07-03 12:17:27
6685896,jPCXuSBbI_cPocgoNBhdkg,V6BjjQICDO4q7TT3ZhaPWw,yA6dKNm_zl1ucZCnwW8ZCg,1.0,2013-05-22 15:32:08
6685897,zwzO3yPdGbdgGy3-XQ0SWg,PFiIECX8wuvi7P-1mCvIjA,a192hdM0_UVCYLwPJv1Qwg,5.0,2018-11-13 21:15:20
6685898,o8Nc2BJhKJXM6tRHJDRuPA,wu5R5_N7q5iqCBh6NBX26Q,kOo4ZY2UQAX4j312mzQ8mA,5.0,2018-08-13 18:09:16


In [4]:
business_id = []
business_name = []
business_city = []
business_state = []
business_catogories = []

for business in businesses:
    business_id.append(business['business_id'])
    business_name.append(business['name'])
    business_city.append(business['city'])
    business_state.append(business['state'])    
    
    categories = None
    if business['categories'] != None:
        categories = ""
        for category in business['categories']:
            categories += category
            
    business_catogories.append(categories)
    
businessDF = pd.DataFrame({'business_id': business_id,
                           'name': business_name,
                             'city': business_city,
                             'state': business_state, 
                          'categories': business_catogories})
businessDF

,business_id,name,city,state,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,Phoenix,AZ,"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,Mississauga,ON,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,Charlotte,NC,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,Goodyear,AZ,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,Charlotte,NC,"Plumbing, Shopping, Local Services, Home Servi..."
...,...,...,...,...,...
192604,nqb4kWcOwp8bFxzfvaDpZQ,Sanderson Plumbing,North Las Vegas,NV,"Water Purification Services, Water Heater Inst..."
192605,vY2nLU5K20Pee-FdG0br1g,Chapters,Newmarket,ON,"Books, Mags, Music & Video, Shopping"
192606,MiEyUDKTjeci5TMfxVZPpg,Phoenix Pavers,Phoenix,AZ,"Home Services, Contractors, Landscaping, Mason..."
192607,zNMupayB2jEHVDOji8sxoQ,Beasley's Barber Shop,Mesa,AZ,"Beauty & Spas, Barbers"


In [ ]:
import math
businessbystate = businessDF.groupby('state') 

categories = ["Afghan",
"African",
"American (New)",
"American (Traditional)",
"Arabian",
"Argentine",
"Armenian",
"Asian Fusion",
"Australian",
"Austrian",
"Bangladeshi",
"Barbeque",
"Basque",
"Belgian",
"Brasseries",
"Brazilian",
"Breakfast & Brunch",
"British",
"Buffets",
"Burgers",
"Burmese",
"Cafes",
"Cafeteria",
"Cajun/Creole",
"Cambodian",
"Caribbean",
"Catalan",
"Cheesesteaks",
"Chicken Shop",
"Chicken Wings",
"Chinese",
"Comfort Food",
"Creperies",
"Cuban",
"Czech",
"Delis",
"Diners",
"Dinner Theater",
"Ethiopian",
"Fast Food",
"Filipino",
"Fish & Chips",
"Fondue",
"Food Court",
"Food Stands",
"French",
"Game Meat",
"Gastropubs",
"German",
"Gluten-Free",
"Greek",
"Guamanian",
"Halal",
"Hawaiian",
"Himalayan/Nepalese",
"Honduran",
"Hong Kong Style Cafe",
"Hot Dogs",
"Hot Pot",
"Hungarian",
"Iberian",
"Indian",
"Indonesian",
"Irish",
"Italian",
"Japanese",
"Kebab",
"Korean",
"Kosher",
"Laotian",
"Latin American",
"Live/Raw Food",
"Malaysian",
"Mediterranean",
"Mexican",
"Middle Eastern",
"Modern European",
"Mongolian",
"Moroccan",
"New Mexican Cuisine",
"Nicaraguan",
"Noodles",
"Pakistani",
"Pan Asia",
"Persian/Iranian",
"Peruvian",
"Pizza",
"Polish",
"Polynesian",
"Pop-Up Restaurants",
"Portuguese",
"Poutineries",
"Russian",
"Salad",
"Sandwiches",
"Scandinavian",
"Scottish",
"Seafood",
"Singaporean",
"Slovakian",
"Soul Food",
"Soup",
"Southern",
"Spanish",
"Sri Lankan",
"Steakhouses",
"Supper Clubs",
"Sushi Bars",
"Syrian",
"Taiwanese",
"Tapas Bars",
"Tapas/Small Plates",
"Tex-Mex",
"Thai",
"Turkish",
"Ukrainian",
"Uzbek",
"Vegan",
"Vegetarian",
"Vietnamese",
"Waffles",
"Wraps"]

rowNames = []
finalDF = []

for name, group in businessbystate:
    rowNames.append(name)
    rowRatings = []
    for cat in categories:
        
        relDataset = group[group['categories'].str.contains(cat)==True]
        relbusinesses = relDataset['business_id']
        
        relRating = ratingsDF[ratingsDF['business_id'].isin(relbusinesses)]
        
        val = relRating['stars'].mean()
        
        if(math.isnan(val)):
            rowRatings.append(0)
        else:
            rowRatings.append(val)

    finalDF.append(rowRatings)

df = pd.DataFrame(finalDF, columns = categories)
df

In [ ]:
df.to_csv(index=False)

In [ ]:
df['state'] = rowNames
df.set_index('state')

In [ ]:
#Get the businesses from Pittsburgh
state_business_ids = []
state_business_names = []
state_business_categories = []

for jsonObj in businesses:
    if jsonObj['state'] == 'PA':
        state_business_ids.append(jsonObj['business_id'])
        state_business_names.append(jsonObj['name'])
        
        categories = None
        if jsonObj['categories'] != None:
            categories = ""
            for category in jsonObj['categories']:
                categories += category
            
        state_business_categories.append(categories)

business = pd.DataFrame({'business_id': state_business_ids,
                             'name': state_business_names,
                             'category': state_business_categories})
print(len(business))
business.head()


In [ ]:
#Get all the review information
user_ids = []
bus_ids = []
ratings = []
dates = []

for jsonObj in reviews:
    user_ids.append(jsonObj['user_id'])
    bus_ids.append(jsonObj['business_id'])
    ratings.append(jsonObj['stars'])
    dateForm = jsonObj['date']
    dateF = datetime.strptime(dateForm, '%Y-%m-%d %H:%M:%S')
    dates.append(dateF)

ratings = pd.DataFrame({'user_id': user_ids,
                        'business_id': bus_ids,
                        'rating': ratings, 
                        'date':dates})

print(len(ratings))
ratings.head()

In [ ]:
#Filter so we only get the information for reviews on Pittsburgh businesses
ratings = ratings[ratings['business_id'].isin(pitt_business_ids)]
print(len(ratings))
ratings.head()


In [ ]:
def create_category_table(categories, column_names):
    category_ratings = pd.DataFrame()
    
    for category in categories:
        cat_businesses = business[business['category'].str.contains(pat=category, na=False)]
        ratings_from_cat = ratings[ratings['business_id'].isin(cat_businesses['business_id'])]
        avg_votes_per_user = ratings_from_cat.loc[:, ['user_id', 'rating']].groupby(['user_id'])['rating'].mean().round(2)
        category_ratings = pd.concat([category_ratings, avg_votes_per_user], axis=1)

    category_ratings.columns = column_names

    return category_ratings

BurgerChineseRatings = create_category_table(['Burgers', 'Chinese'], ['avg_burgers_ratings', 'avg_chinese_ratings'])
BurgerChineseRatings.head()